In [73]:
import numpy as np
import matplotlib.pyplot as pl
import ot
import ot.plot

import os

In [75]:
# SANDBOX

print(os.getcwd())

newdirpath = os.path.join(os.getcwd(), "NewFolder")
os.mkdir(newdirpath)

/Users/NGKarris/Documents/UCSD/Research/LOTLibrary


In [64]:
# (NOT NEEDED NOW) calculates barycentric projection of OT map
#
# x_ref, x_tar : (num,dim) matrices of the reference and target points
# p_ref, p_tar : (num,) probability vectors
# C : ground cost (num_ref,num_tar) matrix

def calc_OT_map(x_ref, x_tar, p_ref=None,p_tar=None,
                C=None, sinkhorn=False, lambd=1, normalize_T=True):
    
    num_ref, dim_ref = x_ref.shape              # number of points, and
    num_tar, dim_tar = x_tar.shape              # dimension of those points
    if p_ref is None:
        p_ref = np.ones((num_ref,))/num_ref     # if not given, assume the
    if p_tar is None:                           # probability density is uniform
        p_tar = np.ones((num_tar,))/num_tar
    
    if C is None:
        assert dim_ref == dim_tar
        C = ot.dist(x_ref, x_tar)               # ground cost is square distance
    
    # G : (num_ref,num_tar) matrix encoding of OT map
    if sinkhorn:
        G = ot.sinkhorn(p_ref, p_tar, C, lambd)
    else:
        G = ot.emd(p_ref, p_tar, C)
    
    
    Gstochastic = G/G.sum(axis=1)[:,None]       # row-normalized G
    T = Gstochastic @ x_tar                     # barycentric projection of G
    if normalize_T:
        T = T/np.sqrt(dim_ref)                  
    return T

In [65]:
# compute_cost

# returns :
    # C : (num_ref,num_tar) ground cost matrix

# parameters:
    # x_ref : (num_ref,dim_ref) num_ref reference points of dimension dim_ref
    # x_tar : (num_tar,dim_tar) num_tar target points of dimension dim_tar
        # if None then x_tar = x_ref
    # metric : method to use to compute costs, e.g.
        # "square_euclidean" , "euclidean" , "discrete" 

def compute_cost(x_ref, x_tar = None, metric = "square_euclidean"):
    
    if x_tar is None:
        x_tar = x_ref

    assert x_ref.shape[1] == x_tar.shape[1]
    
    # c(x,y) = ||x-y||_2^2
    if metric == "square_euclidean":
        C = ot.dist(x_ref, x_tar)
    
    # c(x,y) = ||x-y||_2
    elif metric == "euclidean":
        C = ot.dist(x_ref, x_tar, 'euclidean')
    
    # c(x,y) = {1 if x != y},{0 if x == y}
    elif metric == "discrete":
        num_ref = x_ref.shape[0]
        num_tar = x_tar.shape[0]
        assert num_ref == num_tar
        C = np.ones((num_ref, num_tar)) - np.identity(num_ref)
    
    return C

In [66]:
# create_plan

# returns :
    # G : (num_ref,num_tar) encoding of OT plan

# parameters :
    # p_ref : (num_ref,) reference weights
    # p_tar : (num_ref,) target weights
    # C : (num_ref,num_tar) cost matrix
    # sinkhorn : bool whether to use sinkhorn
    # beta : regularization parameter

def create_plan(p_ref, p_tar, C, sinkhorn=False, beta = 0.01):
    
    if sinkhorn:
        G = ot.sinkhorn(p_ref, p_tar, C, beta)
    else:
        G = ot.emd(p_ref, p_tar, C)
    
    return G

In [67]:
# compute_barycentric_map

# returns :
    # T : (num_ref, num_tar) barycentric projection map

# parameters :
    # G : (num_ref, num_tar) encoding of OT plan
    # x_tar : (num_tar,dim_tar) num_tar target points of dimension dim_tar

def compute_barycentric_map(G, x_tar):
    
    Gstochastic = G/G.sum(axis=1)[:,None]       # row-normalized G
    T = Gstochastic @ x_tar                     # barycentric projection of G
       
    return T

In [68]:
class LOT(object):
    def __init__(self, x_ref, x_tar, p_ref=None, p_tar=None, sinkhorn = False, beta = 0.01, metric = "square_euclidean"):
        self.x_ref = x_ref
        self.x_tar = x_tar

        self.num_ref, self.dim_ref = x_ref.shape
        self.num_tar, self.dim_tar = x_tar.shape
        if p_ref is None:
            self.p_ref = np.ones((self.num_ref,))/self.num_ref
        else:
            self.p_ref = p_ref
        if p_tar is None:
            self.p_tar = np.ones((self.num_tar,))/self.num_tar
        else:
            self.p_tar = p_tar
        
        self.sinkhorn = sinkhorn
        self.beta = beta
        self.metric = metric
        self.cost = None
        self.plan = None
        self.OT_map = None

    def calc_OT_map(self,):
        self.cost = compute_cost(self.x_ref, self.x_tar, self.metric)
        print(self.cost)
        self.plan = create_plan(self.p_ref, self.p_tar, self.cost, self.sinkhorn, self.beta)
        print(self.plan)
        self.OT_map = compute_barycentric_map(self.plan, self.x_tar)

lot1 = LOT(ref, tar, pr, pt, False, 0.01)
print(lot1.x_ref)
print("X_TAR =",lot1.x_tar)
lot1.calc_OT_map()
print("MAP = ", lot1.OT_map)

[[0. 0.]
 [1. 0.]
 [2. 0.]]
X_TAR = [[0. 0.]
 [1. 0.]
 [2. 0.]]
[[0. 1. 4.]
 [1. 0. 1.]
 [4. 1. 0.]]
[[1. 2. 0.]
 [0. 0. 2.]
 [0. 0. 1.]]
MAP =  [[0.66666667 0.        ]
 [2.         0.        ]
 [2.         0.        ]]


In [ ]:
# create_directory

# returns :
    # void

# parameters :
    # base_path : str with path of base directory
    # dir_name : str with name of new directory

def create_directory(base_path, dir_name):
    path = os.path.join(base_path, dir_name)
    try:
        os.mkdir(path)
    except:
        print("Directory Already Exists")